In [ ]:
# SETUP
!pip install -q tensorflow scikit-learn scipy h5py

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model
import os

print(f'TensorFlow: {tf.__version__}')
print(f'GPU: {", ".join([d.name for d in tf.config.list_physical_devices("GPU")]) or "None"}')

In [ ]:
# === ENHANCED DATA GENERATION ===
def generate_enhanced_impact(impact_type):
    """Multi-harmonic physics model"""
    params = {
        'blast': {'amp': (150,300), 'f1': (50,150), 'f2': (100,250), 'decay': (12,18), 'sev': (0.6,1.0)},
        'gunshot': {'amp': (40,100), 'f1': (100,300), 'f2': (200,600), 'decay': (20,30), 'sev': (0.5,0.9)},
        'artillery': {'amp': (200,400), 'f1': (30,100), 'f2': (50,180), 'decay': (10,15), 'sev': (0.6,1.0)},
        'vehicle_crash': {'amp': (20,80), 'f1': (5,30), 'f2': (10,50), 'decay': (3,7), 'sev': (0.3,0.7)},
        'fall': {'amp': (15,60), 'f1': (3,20), 'f2': (5,35), 'decay': (5,12), 'sev': (0.3,0.7)},
        'normal': {'amp': (0.5,3), 'f1': (0.1,5), 'f2': (1,8), 'decay': (1,3), 'sev': (0.0,0.2)}
    }
    p = params[impact_type]
    
    # Multi-harmonic signal
    t = np.linspace(0, 1, 200)
    amp, f1, f2, decay = [np.random.uniform(*p[k]) for k in ['amp','f1','f2','decay']]
    start, dur = np.random.randint(10,50), np.random.randint(30,120)
    
    sig = np.zeros(200)
    impact_t = t[start:start+dur] - t[start]
    sig[start:start+dur] = amp * (
        np.sin(2*np.pi*f1*impact_t) * np.exp(-decay*impact_t) +
        0.3 * np.sin(2*np.pi*f2*impact_t) * np.exp(-decay*1.2*impact_t)
    ) + np.random.normal(0, amp*0.1, dur)
    
    # 3-axis IMU
    accel_z = sig
    accel_x = np.roll(sig, 5) * np.random.uniform(0.2,0.5) + np.random.normal(0, 0.5, 200)
    accel_y = np.roll(sig, 3) * np.random.uniform(0.2,0.5) + np.random.normal(0, 0.5, 200)
    
    gyro_x = np.gradient(accel_y) * 12 + np.random.normal(0, 2, 200)
    gyro_y = np.gradient(accel_x) * 12 + np.random.normal(0, 2, 200)
    gyro_z = np.gradient(accel_z) * 12 + np.random.normal(0, 2, 200)
    
    mag = np.tile([30,20,-40], (200,1)) + np.random.normal(0, 3, (200,3))
    
    # Enhanced vitals
    sev = np.random.uniform(*p['sev'])
    hr_base = np.random.uniform(60, 80)
    hr = hr_base + sev*90*(1-np.exp(-np.arange(200)/100)) + 3*np.sin(2*np.pi*0.25*t) + np.random.normal(0,2,200)
    hr = np.clip(hr, 40, 180)
    
    spo2 = 97 - sev*18*(1-np.exp(-np.arange(200)/180)) + np.random.normal(0,0.5,200)
    spo2 = np.clip(spo2, 70, 100)
    
    br = 14 + sev*15*(1-np.exp(-np.arange(200)/120)) + np.random.normal(0,1,200)
    br = np.clip(br, 8, 40)
    
    temp = 36.6 + (sev*0.5 if sev<0.7 else -sev) * (1-np.exp(-np.arange(200)/250)) + np.random.normal(0,0.08,200)
    temp = np.clip(temp, 34, 39)
    
    return np.column_stack([accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, mag, hr, spo2, br, temp]).astype(np.float32), sev

# Generate 60K samples
print('Generating 60,000 samples...')
CLASSES = ['blast','gunshot','artillery','vehicle_crash','fall','normal']
X, y_type, y_sev = [], [], []

for i, cls in enumerate(CLASSES):
    print(f'{cls}...', end=' ')
    for _ in range(10000):
        s, sev = generate_enhanced_impact(cls)
        X.append(s)
        label = np.zeros(6); label[i] = 1
        y_type.append(label)
        y_sev.append(sev)
    print('✓')

X = np.array(X, dtype=np.float32)
y_type = np.array(y_type, dtype=np.float32)
y_sev = np.array(y_sev, dtype=np.float32)

print(f'\nDataset shape: {X.shape}')

In [ ]:
# Normalize
print('Normalizing features...')
X_mean, X_std = X.mean(axis=(0,1), keepdims=True), X.std(axis=(0,1), keepdims=True) + 1e-8
X = (X - X_mean) / X_std

# Split
from sklearn.model_selection import train_test_split
X_train, X_temp, y_type_train, y_type_temp, y_sev_train, y_sev_temp = train_test_split(
    X, y_type, y_sev, test_size=0.3, random_state=42, stratify=y_type.argmax(1))
X_val, X_test, y_type_val, y_type_test, y_sev_val, y_sev_test = train_test_split(
    X_temp, y_type_temp, y_sev_temp, test_size=0.5, random_state=42, stratify=y_type_temp.argmax(1))

print(f'Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}')

In [ ]:
# === ADVANCED MODEL ===
def res_block(x, f, d=1):
    s = x
    x = layers.Conv1D(f, 3, padding='same', dilation_rate=d)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv1D(f, 3, padding='same', dilation_rate=d)(x)
    x = layers.BatchNormalization()(x)
    if s.shape[-1] != f: s = layers.Conv1D(f, 1)(s)
    return layers.ReLU()(layers.Add()([s, x]))

inputs = layers.Input((200, 13))
x = layers.Conv1D(64, 7, padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Residual blocks with dilations
x = res_block(x, 64, 1)
x = res_block(x, 64, 2)
x = layers.MaxPooling1D(2)(x)

x = res_block(x, 128, 1)
x = res_block(x, 128, 2)
x = res_block(x, 128, 4)
x = layers.MaxPooling1D(2)(x)

x = res_block(x, 256, 1)
x = res_block(x, 256, 2)
x = res_block(x, 256, 4)

x = res_block(x, 512, 1)
x = res_block(x, 512, 2)

# Multi-head attention
attn = layers.MultiHeadAttention(16, 32)(x, x)
x = layers.LayerNormalization()(layers.Add()([x, attn]))

# SE block
se = layers.GlobalAveragePooling1D()(x)
se = layers.Dense(32, activation='relu')(se)
se = layers.Dense(512, activation='sigmoid')(se)
x = layers.Multiply()([x, layers.Reshape((1, 512))(se)])

# Multi-scale pooling
avg = layers.GlobalAveragePooling1D()(x)
mx = layers.GlobalMaxPooling1D()(x)
attn_w = layers.Dense(1, activation='softmax')(x)
attn_p = layers.Flatten()(layers.Dot(1)([layers.Permute((2,1))(x), attn_w]))
x = layers.concatenate([avg, mx, attn_p])

# Deep head
x = layers.Dense(768, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

impact = layers.Dense(6, activation='softmax', name='impact_type')(x)
severity = layers.Dense(1, activation='sigmoid', name='severity')(x)

model = Model(inputs, [impact, severity])
model.compile(
    optimizer=tf.keras.optimizers.AdamW(0.001, weight_decay=0.0001, clipnorm=1.0),
    loss={'impact_type': tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), 'severity': 'mse'},
    loss_weights={'impact_type': 1.0, 'severity': 0.2},
    metrics={'impact_type': ['accuracy'], 'severity': ['mae']}
)

print(f'\nParameters: {sum([np.prod(v.shape) for v in model.trainable_weights]):,}')
model.summary()

In [ ]:
# === TRAIN WITH AUGMENTATION ===
@tf.function
def augment(x):
    # Time shift
    x = tf.roll(x, tf.random.uniform((), -10, 10, tf.int32), 0)
    # Scale
    x *= tf.random.uniform((), 0.9, 1.1)
    # Noise
    x += tf.random.normal(tf.shape(x), 0, 0.03)
    return x

def data_gen(X, y_type, y_sev, batch=32, aug=False):
    while True:
        idx = np.random.permutation(len(X))
        for i in range(0, len(X), batch):
            batch_idx = idx[i:i+batch]
            X_b = X[batch_idx]
            if aug:
                X_b = np.array([augment(x).numpy() for x in X_b])
            yield X_b, {'impact_type': y_type[batch_idx], 'severity': y_sev[batch_idx]}

train_gen = data_gen(X_train, y_type_train, y_sev_train, aug=True)
val_gen = data_gen(X_val, y_type_val, y_sev_val, aug=False)

callbacks = [
    tf.keras.callbacks.EarlyStopping('val_impact_type_accuracy', patience=30, restore_best_weights=True, mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau('val_loss', 0.5, patience=12, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', 'val_impact_type_accuracy', save_best_only=True, mode='max'),
    tf.keras.callbacks.LearningRateScheduler(lambda e: 0.001 * (np.cos(np.pi*e/200)+1)/2)
]

print('\nTraining...')
print('='*70)

history = model.fit(
    train_gen, steps_per_epoch=len(X_train)//32,
    validation_data=val_gen, validation_steps=len(X_val)//32,
    epochs=200, callbacks=callbacks, verbose=1
)

In [ ]:
# === EVALUATE ===
print('\n' + '='*70)
print('FINAL EVALUATION')
print('='*70)

results = model.evaluate(X_test, {'impact_type': y_type_test, 'severity': y_sev_test}, verbose=1)
test_acc = results[3]

print(f'\n{"="*70}')
print(f'TEST ACCURACY: {test_acc:.4f} ({test_acc*100:.2f}%)')
print(f'TARGET: 0.9600 (96.00%)')
if test_acc >= 0.96: 
    print('✓ TARGET ACHIEVED!')
else: 
    print(f'✗ Below by {(0.96-test_acc)*100:.2f}%')
print('='*70)

In [ ]:
# === CONVERT TO TFLITE ===
print('\nConverting to TFLite...')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

try:
    tflite_model = converter.convert()
    with open('impact_classifier.tflite', 'wb') as f:
        f.write(tflite_model)
    print(f'✓ TFLite saved ({len(tflite_model)/1024:.1f} KB, accuracy: {test_acc:.4f})')
except Exception as e:
    print(f'⚠ TFLite conversion failed: {e}')
    model.save('model.h5')
    print('✓ Saved as model.h5 instead')

In [ ]:
# === DOWNLOAD FILES ===
try:
    from google.colab import files
    
    if os.path.exists('impact_classifier.tflite'):
        files.download('impact_classifier.tflite')
        print('✓ Downloaded impact_classifier.tflite')
    elif os.path.exists('model.h5'):
        files.download('model.h5')
        print('✓ Downloaded model.h5')
    
    if os.path.exists('best_model.h5'):
        files.download('best_model.h5')
        print('✓ Downloaded best_model.h5')
    
    print('\n' + '='*70)
    print('DEPLOYMENT READY')
    print('='*70)
except:
    print('Not running in Colab - files saved locally')